# 1. Generate Data

In [1]:
N_JOBS = 7
N_MACHINES = 4

In [2]:
import random
# The processing time of a job at a machine is defined in a matrix proc[job][machine]
def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
# Initialisation code base on the last 4 digits of Student ID: 2312 7635
PROC = generateData(7635)

In [4]:
PROC

[[3, 6, 7, 3],
 [9, 1, 4, 8],
 [3, 5, 1, 3],
 [5, 5, 3, 9],
 [5, 1, 8, 5],
 [5, 9, 9, 3],
 [2, 6, 1, 7]]

In [5]:
str(PROC)

'[[3, 6, 7, 3], [9, 1, 4, 8], [3, 5, 1, 3], [5, 5, 3, 9], [5, 1, 8, 5], [5, 9, 9, 3], [2, 6, 1, 7]]'

# 2. Job Scheduling using Greedy Algorithm

In [6]:
def greedy_sequence(PROC):
    job_sequence = []
    selected_jobs = set()
    
    for machine in range (N_MACHINES):
        job_time= [(PROC[job][machine], job) 
                   for job in range(N_JOBS) if job not in selected_jobs]

        job_time.sort()
        if job_time:
            _, selected_job = job_time[0]
            job_sequence.append(selected_job)
            selected_jobs.add(selected_job)
    for job in range (N_JOBS):
        if job not in selected_jobs:
            job_sequence.append(job)
             
    return job_sequence

In [21]:
def optimal_time(PROC, job_sequence):
        
    idle_time = [[0]* N_MACHINES for _ in range(N_JOBS)]
    start_time= [[0]* N_MACHINES for _ in range(N_JOBS)]
    all_time = [[0]* N_MACHINES for _ in range(N_JOBS)]
    stop_time = [[0]* N_MACHINES for _ in range(N_JOBS)]
    wait_time = [[0]* N_MACHINES for _ in range(N_JOBS)]

    for job_index in range(N_JOBS):
       
        job = sequence[job_index]
        for machine in range (N_MACHINES):
            wait_time[0][machine]= 0
            if job_index == 0 and machine == 0:
                #Starting time on the first MACHINE
                start_time[job][machine] = 0
            elif job_index == 0:
                start_time[job][machine] = all_time[job][machine-1]
            elif machine ==0:
                start_time[job][machine] = all_time[sequence[job_index-1]][machine]
            else:
                start_time[job][machine]= max(all_time[job][machine-1],
                                           all_time[sequence[job_index-1]][machine])

            # Compute idle time for the machine
            if job_index > 0:
                idle_time[job][machine] = max(0, start_time[job][machine] - all_time[job_sequence[job_index-1]][machine])

            # Compute wait time for the job
            if machine > 0:
                wait_time[job][machine] = start_time[job][machine] - stop_time[job][machine-1]

           # idle_time[job][machine] = max(0, start_time[job][machine]- all_time[sequence[job_index-1]][machine])
           # wait_time[job][machine] = start_time[job][machine]- stop_time[job-1][machine]
            all_time[job][machine] = start_time[job][machine]+PROC [job][machine]
            stop_time[job][machine] = all_time[job][machine]
    last_job = sequence[-1]
    
    total_time = all_time[last_job][N_MACHINES-1]
    return total_time, start_time, idle_time, stop_time, wait_time

In [22]:
sequence = greedy_sequence(PROC)

In [23]:
sequence

[6, 1, 2, 0, 3, 4, 5]

In [24]:
total_time, start_time, idle_time, stop_time, wait_time = optimal_time(PROC, sequence)

In [25]:
start_time

[[14, 19, 25, 32],
 [2, 11, 12, 16],
 [11, 14, 19, 24],
 [17, 25, 32, 35],
 [22, 30, 35, 44],
 [27, 32, 43, 52],
 [0, 2, 8, 9]]

In [26]:
stop_time

[[17, 25, 32, 35],
 [11, 12, 16, 24],
 [14, 19, 20, 27],
 [22, 30, 35, 44],
 [27, 31, 43, 49],
 [32, 41, 52, 55],
 [2, 8, 9, 16]]

In [27]:
wait_time

[[0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 4],
 [0, 3, 2, 0],
 [0, 3, 4, 1],
 [0, 0, 2, 0],
 [0, 0, 0, 0]]

In [28]:
idle_time

[[0, 0, 5, 5],
 [0, 3, 3, 0],
 [0, 2, 3, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 1, 0, 3],
 [0, 0, 0, 0]]

# 3. Structure of the schedule

In [29]:
def schedule(sequence, N_JOBS, N_MACHINES, idle_time, start_time, wait_time, stop_time):
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
               f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {(idle_time[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{(start_time[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {j+1:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {(wait_time[j][m]):4d} | " \
                   f"Proc: {(PROC[j][m]):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {(stop_time[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    return row

The processing times per job and machine:

In [30]:
PROC

[[3, 6, 7, 3],
 [9, 1, 4, 8],
 [3, 5, 1, 3],
 [5, 5, 3, 9],
 [5, 1, 8, 5],
 [5, 9, 9, 3],
 [2, 6, 1, 7]]

The optimal job schedule:

In [31]:
print(sequence)

[6, 1, 2, 0, 3, 4, 5]


The processing time of the optimal job schedule:

In [32]:
stop_time

[[17, 25, 32, 35],
 [11, 12, 16, 24],
 [14, 19, 20, 27],
 [22, 30, 35, 44],
 [27, 31, 43, 49],
 [32, 41, 52, 55],
 [2, 8, 9, 16]]

In [33]:
print(total_time)

55


The optimal job schedule in detail:

In [35]:
print(schedule(sequence, N_JOBS, N_MACHINES, idle_time, start_time, wait_time, stop_time))

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    5 |            | Idle:    5 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:  14 |            | Start:  19 |            | Start:  25 |            | Start:  32 |
| Job:  1 | Wait:    0 | Proc:    3 | Wait:    0 | Proc:    6 | Wait:    0 | Proc:    7 | Wait:    0 | Proc:    3 |
|         |            | Stop:   17 |            | Stop:   25 |            | Stop:   32 |            | Stop:   35 |
|---------|------------|------------|------------|------------|---------